In [1]:
import torch
from torchvision import datasets, transforms
import numpy as np
import attacks
import blackbox
import pandas as pd
# import utils
from tqdm.notebook import trange
torch.manual_seed(0)
import torch.nn as nn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import confusion_matrix

# dataloaders, dataset_sizes = utils.task_loader('online')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 
device

device(type='cuda', index=0)

In [3]:
mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor() )
# create_task(mnist_train,target,[0.5,0.5], 'train')

mnist_val = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor() )
# create_task(mnist_val,target,[0.5,0.5], 'val')

In [27]:
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset

class DataStream:

    def __init__(self, ):
        self.R = {}
        self.Sa = []

        
    def random_stream(self, data, model, outcome_distribution ):

        stream = []
        label = []
        outcomes = []
        for i in tqdm( range( len(data) ) ): #
            outcome = np.random.choice( [1, 0], p=outcome_distribution)
            X,y = data[i]
            X = X.reshape( (1,1,28,28) ).to(device)
            y = torch.Tensor([ y ]).type(torch.LongTensor).to(device)
            if outcome==1:
                delta = attacks.pgd_linf(model, X , y).to(device)
                X = X + delta
            stream.append(  X[0] )
            label.append(y)
            outcomes.append([outcome])
        stream = torch.stack(stream)
        label = torch.stack(label)
        status = torch.Tensor(outcome)
        
        dataset = TensorDataset( stream, label, status )

        return dataset

    def virtual_plus(self, N, i, v_i, t = 25, k = 10, exhaust = True):

        outcome = 0

        if i <= t: # sampling phase
            self.R[i] = v_i
            self.R = {k: v for k, v in sorted(self.R.items(), key=lambda item: item[1])}
            
        else: # selection phase
            last_value = list( self.R.values() )[-1]
            last_key = list( self.R.keys() )[-1]
            num_left_to_pick = k - len(self.Sa) 
            num_samples_left = N - i
            if  num_samples_left <= num_left_to_pick and exhaust and num_left_to_pick > 0:
                self.Sa.append(i)
                outcome = 1

            if v_i >= last_value and len(self.Sa) <= k:
                del self.R[ last_key ]
                self.R[i] = v_i
                self.R = {k: v for k, v in sorted(self.R.items(), key=lambda item: item[1])}
                self.Sa.append(i)
                outcome = 1

        return outcome

        

    def secretary_stream(self, data, model):

        stream = []
        label = []
        outcomes = []

        N = 100 #len(data)
        for i in tqdm( range( N ) ): 

            X, y = data[i]
            X = X.reshape( (1,1,28,28) ).to(device)
            y = torch.Tensor([ y ]).type(torch.LongTensor).to(device)

            delta = attacks.pgd_linf(model, X , y).to(device)
            X_prime = X + delta
            v_i = nn.CrossEntropyLoss()( model(X_prime) ,y)

            outcome = self.virtual_plus(N, i, v_i)

            stream.append(  X[0] )
            label.append(y)
            outcomes.append([outcome])

        stream = torch.stack(stream)
        label = torch.stack(label)
        status = torch.Tensor(outcomes)
        
        dataset = TensorDataset( stream, label, status )

        return dataset



blackbox_model = blackbox.load()
mnist_val = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor() )
datastream = DataStream()
# random_stream = datastream.random_stream( mnist_val, blackbox_model, [0.5,0.5] )
sec_stream = datastream.secretary_stream( mnist_val, blackbox_model  )

  0%|          | 0/100 [00:00<?, ?it/s]

In [28]:
k = 0
for batch, data in enumerate(sec_stream):
    
    X,label,outcome = data
    k += outcome

k

tensor([10.])